In [ ]:

seed_value= 0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)


import tensorflow as tf
#tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)

from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain 

from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers

In [ ]:
train_dir = os.path.join('Reliance', 'Train') #Just the directory of training_images
validation_dir = os.path.join('Reliance', 'Validation') #Just the directory of valiadtion_images

In [ ]:
df2=pd.read_csv("testing.csv")  #test-set  for financial evaluation

In [ ]:
train_buy_dir = os.path.join(train_dir, 'Buy')  # directory with our training  Buy pictures
train_hold_dir = os.path.join(train_dir, 'Hold')  # directory with our training Hold pictures
train_sell_dir = os.path.join(train_dir, 'Sell')  # directory with our training Sell pictures
validation_buy_dir = os.path.join(validation_dir, 'Buy')  # directory with our validation Buy pictures
validation_hold_dir = os.path.join(validation_dir, 'Hold')  # directory with our validation Hold pictures
validation_sell_dir = os.path.join(validation_dir, 'Sell')  # directory with our validation Sell pictures

num_buy_tr = len(os.listdir(train_buy_dir))
num_hold_tr = len(os.listdir(train_hold_dir))
num_sell_tr = len(os.listdir(train_sell_dir))

num_buy_val = len(os.listdir(validation_buy_dir))
num_hold_val = len(os.listdir(validation_hold_dir))
num_sell_val = len(os.listdir(validation_sell_dir))

total_train = num_buy_tr + num_hold_tr + num_sell_tr
total_val = num_buy_val + num_hold_val + num_sell_val



print("Total training images:", total_train)
print("Total validation images:", total_val)

In [ ]:
batch_size = 32
epochs = 100
IMG_HEIGHT = 12
IMG_WIDTH = 15

# ***Data Augmentation***

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#tf.keras.preprocessing.image_dataset_from_directory(train_dir,)
image_gen_train = ImageDataGenerator(
    rescale=1./255,featurewise_center=True,
    
    width_shift_range=0.1,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='reflect',
    )

validation_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen =image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,color_mode="rgb",
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='sparse')


In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,color_mode="rgb",
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='sparse')

# ***Model***

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
model = keras.Sequential(
    [
        keras.Input(shape=(12,15,3)),
        layers.Conv2D(32,kernel_size=(3, 3), activation="relu"),
        layers.Conv2D(64,kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.25,seed=0),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(3, activation="softmax"),
    ]
)

model.summary()
model.compile(optimizer='rmsprop',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])


# ***Computational Performance***

In [ ]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

In [ ]:
val_loss,val_acc =model.evaluate(val_data_gen,steps=len(test_data_gen),verbose=0)
val_acc

In [ ]:
plt.plot(history.history['accuracy'], label='acc')
plt.plot(history.history['val_accuracy'], label = 'val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.2, 1])
plt.legend(loc='lower right')

In [ ]:
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
                                                 directory='Testing',
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='sparse')

In [ ]:
test_loss,test_acc =model.evaluate(test_data_gen,steps=len(test_data_gen),verbose=0)
test_acc

## ***Financial Evaluation***

In [ ]:
from keras.preprocessing import image
labels_pred=[]

PATH = os.getcwd()
data_path =  'Testing'
data_dir_list = os.listdir(data_path)

img_data_list=[]
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    for img in img_list:
        img_path = data_path + '/'+ dataset + '/'+ img
        img = keras.preprocessing.image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
        test_img = image.img_to_array(img)/255
        test_img=np.array(test_img)
        img_array = tf.expand_dims(test_img, 0)  # Create batch axis
        labels_pred.append(np.argmax(model.predict(img_array),axis=1)[0])
        
print(labels_pred)

In [ ]:
df2['prediction']=labels_pred
icost=[]
buy_l=[]
cost=0
buy_c=0
buy=0
sell=0
win=0
loss=0

for i in df2.index:
    if(df2['prediction'][i]==0):
        if(buy_c<=0):
            buy_c=buy_c+1
            buy=buy+1
            cost=cost - df2['Adj Close'][i]
            x=df2['Adj Close'][i]
            #print(cost)
            buy_l.append(x)
            print("date : {}, cost : {},buy_c:{},price:{}".format(i, cost,buy_c,x),'Buy')
    if(df2['prediction'][i]==2):
        y=df2['Adj Close'][i]
        if(buy_c>=1):
            if(y-x>0):
                buy_c=buy_c-1
                sell=sell+1
                cost=cost+df2['Adj Close'][i]
                print("date : {}, cost :{},buy_c:{},price:{}".format(i, cost,buy_c,y),"Sell")
                if((buy>0) & (sell>0)):
                    if(y-x<0):
                        loss=loss+1
                    if(y-x>0):
                        win=win+1
                    icost.append(y-x)
    if(df2['prediction'][i]==1):
        continue
    
    
    
  
print(cost)
#print((cost-icost)/icost)
print(buy)
print(sell)
print("win:",win)
print('loss:',loss)